# Campaign Finance Data

This is a IPython Jupyter notebook for accessing US Federal Election Commission data from 1980 until 2016 (as of this writing)

The URL for the data dictionary is here: http://www.fec.gov/finance/disclosure/ftpdet.shtml

The contents has also been captured locally to protect against changes in future availability.

#### Committees (Committee Master)
The committee master file contains one record for each committee registered with the Federal Election Commission. This includes federal political action committees and party committees, campaign committees for presidential, house and senate candidates, as well as groups or organizations who are spending money for or against candidates for federal office.
The file contains basic information about the committees. The ID number the Commission assigned to the committee is first, along with the name of the committee, the sponsor, where appropriate, the treasurer's name and the committee's address. The file also includes information about what type of committee is being described, along with the candidate's ID number if it is a campaign committee. A comma delimited header file is available on this file's data dictionary page.

#### Candidates (Candidate Master)
The candidate master file contains one record for each candidate who has either registered with the Federal Election Commission or appeared on a ballot list prepared by a state elections office.
The file contains basic information about the candidate, including name, party, whether the candidate is an incumbent, challenger, or involved in an open seat, address, state and district in which the candidate is running and the year of the election for which the candidate is registered. (Note that incumbent/challenger status is dynamic in the current election cycle and there may be delays in identifying districts that will involve open seats. The file also includes the ID number assigned to the candidate by the FEC which is used in tracking campaign finance information about the campaign, as well as the ID number of the candidate's principal campaign committee. A comma delimited header file is available on this file's data dictionary page.

#### Linkages (Candidate-Committee Linkage)
This file contains one record for each candidate to committee linakge.
Itemized Records (Any Transaction from One Committee to Another)
The itemized records (miscellaneous transactions) file contains all transactions (contributions, transfers, etc. among federal committees). It contains all data in the itemized committee contributions file plus PAC contributions to party committees, party transfers from state committee to state committee, and party transfers from national committee to state committee. Note that this file only includes federal transfers not soft money transactions. A comma delimited header file is available on this file's data dictionary page.

#### Contributions to Candidates (Contributions to Candidates (and other expenditures) from Committees)
This file is a subset of the Itemized Records (OTH) file. The itemized committee contributions file contains each contribution or independent expenditure made by a PAC, party committee, candidate committee, or other federal committee to a candidate during the two-year election cycle. It includes the ID number of the contributing committee and the ID number of the recipient. You will need to use the committee master and candidate master files in conjunction with this file to set up a relational database to analyze these data. A comma delimited header file is available on this file's data dictionary page.

#### Individual Contributions (Contributions by Individuals)
The individual contributions file contains each contribution from an individual to a federal committee. It includes the ID number of the committee receiving the contribution, the name, city, state, zip code, and place of business of the contributor along with the date and amount of the contribution. NOTE: this file can be very large file. A comma delimited header file is available on this file's data dictionary page.

#### Operating Expenditures
The Operating Expenditures file contains disbursements reported on FEC Form 3 Line 17, FEC Form 3P Line 23and FEC Form 3X Lines 21(a)(i), 21(a)(ii) and 21(b). Operating expenditures disclosed by electronic filers are available for the current election cycle and for election cycles through 2004. Operating expenditures disclosed by paper filers, excluding Form 3P, are available for the current election cycle and for election cycles through 2006. Please note, operating expenditures disclosed by paper filers during the 2006 cycle are only available from reports filed in October 2005 and later. NOTE: this file can be very large file. A comma delimited header file is available on this file's data dictionary page.

In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
from os import environ

In [2]:
downloadDirectory = "~/Desktop/campaignFinance"

years = xrange(1980, 2017, 2)
types = ["cm", "cn", "ccl", "oth", "pas", "indiv", "oppexp"]
url = "ftp://ftp.fec.gov/FEC/{Y}/{t}{y}.zip"

In [7]:
import StringIO
import gzip

def getDataFrame(dtype, year, cols):
    furl = url.format(Y=year, y=str(year)[-2:], t=dtype)
    text = !curl -s $furl | bsdtar -xO -
    df = pd.read_csv(StringIO.StringIO("\n".join(text)), sep="|", engine="c",
                     header=None, error_bad_lines=False, names=cols)
    return df

def saveData(dtype, year, name, columns, compress=True, cleanup=True):
    furl = url.format(Y=year, y=str(year)[-2:], t=dtype)
    header = "|".join(columns)
      
    cleanupSteps = " "
    if cleanup:
        # Workaround for sed "RE error: illegal byte sequence - likely due to WINDOWS-1251 encoding"
        environ['LC_CTYPE'] = 'C'
        environ['LANG'] = 'C'

        # This isn't used at the moment but a few files are WINDOWS 1251 (oppexp-2016, indiv-2010/2012)
        ensureEncoding = "iconv -t UTF-8//IGNORE"
        
        # Need to repeat NA pattern twice - sed doesn't support lookahead
        naReplace = ["s/|[Nn][Aa]|/||/g"]
        fatFingerInt = ["s/|O|/|0|/g"]
        cleanupSteps = "| sed -e '{patterns}'".format(patterns=";".join(naReplace*2 + fatFingerInt)) 
        
    commandBase = "{{ echo '{h}' ; curl -s {u} | bsdtar -xO - {c} ; }} ".format(h=header, u=furl, c=cleanupSteps)
    
    if compress:
        targetFile = downloadDirectory + "/{0}.csv.gz".format("-".join([name, str(year)]))
        command = commandBase + " | gzip > {file}".format(file=targetFile)
        !$command

    else:
        targetFile = downloadDirectory + "/{0}.csv".format("-".join([name, str(year)]))
        command = commandBase + " > {file}".format(file=targetFile)
        !$command
            

committeeMaster = pd.DataFrame(columns=['Committee Identification', 
                                        'Committee Name', 
                                        'Treasurers Name', 
                                        'Street One', 
                                        'Street Two',
                                        'City or Town', 
                                        'State', 
                                        'Zip', 
                                        'Committee Designation',
                                        'Committee Type', 
                                        'Committee Party', 
                                        'Filing Frequency',
                                        'Interest Group Category', 
                                        'Connected Organizations Name',
                                        'Candidate Identification'])

candidateMaster = pd.DataFrame(columns=['Candidate Identification',
                                        'Candidate Name',
                                        'Party Affiliation',
                                        'Year of Election',
                                        'Candidate State',
                                        'Candidate Office',
                                        'Candidate District',
                                        'Incumbent Challenger Status',
                                        'Candidate Status',
                                        'Principal Campaign Committee',
                                        'Mailing Address - Street',
                                        'Mailing Address - Street2',
                                        'Mailing Address - City',
                                        'Mailing Address - State',
                                        'Mailing Address - Zip Code'])

candidateCommitteeLinkage = pd.DataFrame(columns=['Candidate Identification',
                                                  'Candidate Election Year',
                                                  'FEC Election Year',
                                                  'Committee Identification',
                                                  'Committee Type',
                                                  'Committee Designation',
                                                  'Linkage ID'])

interCommitteeTransactions = pd.DataFrame(columns=['Filer Identification Number',
                                                   'Amendment Indicator',
                                                   'Report Type',
                                                   'Primary-General Indicator',
                                                   'Image Number',
                                                   'Transaction Type',
                                                   'Entity Type',
                                                   'Contributor/Lender/Transfer Name',
                                                   'City/Town',
                                                   'State',
                                                   'Zip Code',
                                                   'Employer',
                                                   'Occupation',
                                                   'Transaction Date(MMDDYYYY)',
                                                   'Transaction Amount',
                                                   'Other Identification Number',
                                                   'Transaction ID',
                                                   'File Number / Report ID',
                                                   'Memo Code',
                                                   'Memo Text',
                                                   'FEC Record Number'])

committeeToCandidate = pd.DataFrame(columns=['Filer Identification Number',
                                             'Amendment Indicator',
                                             'Report Type',
                                             'Primary-General Indicator',
                                             'Image Number',
                                             'Transaction Type',
                                             'Entity Type',
                                             'Recipient/Payee',
                                             'City/Town',
                                             'State',
                                             'Zip Code',
                                             'Employer',
                                             'Occupation',
                                             'Transaction Date(MMDDYYYY)',
                                             'Transaction Amount',
                                             'Other Identification Number',
                                             'Candidate Identification Number',
                                             'Transaction ID',
                                             'File Number / Report ID',
                                             'Memo Code',
                                             'Memo Text',
                                             'FEC Record Number'])

individualContributions = pd.DataFrame(columns=['Filer Identification Number',
                                                'Amendment Indicator',
                                                'Report Type',
                                                'Primary-General Indicator',
                                                'Image Number',
                                                'Transaction Type',
                                                'Entity Type',
                                                'Contributor/Lender/Transfer Name',
                                                'City/Town',
                                                'State',
                                                'Zip Code',
                                                'Employer',
                                                'Occupation',
                                                'Transaction Date(MMDDYYYY)',
                                                'Transaction Amount',
                                                'Other Identification Number',
                                                'Transaction ID',
                                                'File Number / Report ID',
                                                'Memo Code',
                                                'Memo Text',
                                                'FEC Record Number'])

operatingExpenditures = pd.DataFrame(columns=['Filer Identification Number',
                                              'Amendment Indicator',
                                              'Report Year',
                                              'Report Type',
                                              'Image Number',
                                              'Line Number',
                                              'Form Type',
                                              'Schedule Type',
                                              'Contributor/Lender/Transfer Name',
                                              'City/Town',
                                              'State',
                                              'Zip Code',
                                              'Transaction Date(MMDDYYYY)',
                                              'Transaction Amount',
                                              'Primary General Indicator',
                                              'Purpose',
                                              'Disbursement Category Code',
                                              'Disbursement Category Code Description',
                                              'Memo Code',
                                              'Memo Text',
                                              'Entity Type',
                                              'FEC Record Number',
                                              'File Number / Report ID',
                                              'Transaction ID',
                                              'Back Reference Transaction ID',
                                              "Null"])



In [10]:
%%time

# REDO 
# oppexp - operatingExpenditures-2016.csv.gz
# indiv - individualContributions-2010.csv.gz
# indiv - individualContributions-2012.csv.gz

saveData("indiv", 2012, "indiv", individualContributions.columns, compress=False, cleanup=False)

#getDataFrame("cm", 2016, committeeMaster.columns).head()

CPU times: user 5.2 s, sys: 1.74 s, total: 6.94 s
Wall time: 5min 10s


# Download the data locally

Note: candidateCommitteeLinkage files only exist after year 2000 and operatingExpenditures after 2004

In [ ]:
%%time
# Total time was ~90 minutes on a 40Mbit connection  (2.9GB)

flags = {"compress": True, "cleanup": True}

for y in years:
    saveData("cm", y, "committeeMaster", committeeMaster.columns, **flags)
    saveData("cn", y, "candidateMaster", candidateMaster.columns, **flags)

    if y >= 2000:
        saveData("ccl", y, "candidateCommitteeLinkage", candidateCommitteeLinkage.columns, **flags)
    
    # This can be on the order of ~100MB
    saveData("oth", y, "interCommitteeTransactions", interCommitteeTransactions.columns, **flags)
    
    saveData("pas2", y, "committeeToCandidate", committeeToCandidate.columns, **flags)
    
    saveData("indiv", y, "individualContributions", individualContributions.columns, **flags)

    if y >= 2004:
        saveData("oppexp", y, "operatingExpenditures", operatingExpenditures.columns, **flags)


# Sample load into a local PostgreSQL database

In [ ]:
"""
# Load Operating Expenses
$ for f in `ls operating*` ; do 
    echo ${f} ; gzcat ${f} | psql -d fec -c "COPY operating_expenses FROM stdin HEADER DELIMITER '|' CSV ENCODING 'windows-1251';" ; 
  done ;

# Load Individual Contributions
$ for f in `ls indiv*` ; do 
    echo ${f} ; 
    gzcat ${f} | psql -d fec -c "COPY individual_contributions FROM stdin HEADER DELIMITER '|' CSV QUOTE E'\b' ;" ;
  done ;

"""

# Use Dask for querying files directly (work in progess)
Note: this is significantly slower than using PostgreSQL on my machine

In [ ]:
base = !ls -d ~
fullDownloadDir = downloadDirectory.replace("~", base[0])

sqlFriendlyNames = lambda a: map(lambda t : t.replace(" ", "_")\
                                             .replace("/", "_or_")\
                                             .replace("-", "_")\
                                             .replace("(MMDDYYYY)", "")\
                                             .lower(), 
                                 a)

committeeMasterDtype = zip(sqlFriendlyNames(committeeMaster.columns), ["str"]*len(committeeMaster.columns))
cmdf = dd.read_csv(fullDownloadDir + "/committeeMaster*.csv*", sep="|", 
                   dtype=committeeMasterDtype,
                   error_bad_lines=False)

cntypes = ["str"]*len(candidateMaster.columns)
# Year of Election
cntypes[3] = "int"
candidateMasterDtype = zip(sqlFriendlyNames(candidateMaster.columns), cntypes)
cndf = dd.read_csv(fullDownloadDir + "/candidateMaster*.csv*", sep="|", 
                   dtype=candidateMasterDtype,
                   error_bad_lines=False)

ccltypes = ["str"]*len(candidateCommitteeLinkage.columns)
# Candidate Election Year
ccltypes[1] = "int"

# FEC Election Year
ccltypes[2] = "int"

# Linkage ID
ccltypes[-1] = "int"
candidateCommitteeLinkageDtype = zip(sqlFriendlyNames(candidateMaster.columns), ccltypes)
ccldf = dd.read_csv(fullDownloadDir + "/candidateCommitteeLinkage*.csv*", sep="|", 
                    dtype=candidateCommitteeLinkageDtype,
                    error_bad_lines=False)


othtypes = ["str"]*len(interCommitteeTransactions.columns)
# Transaction Amount
othtypes[-7] = "float"

# File Number / Report ID - raw data contains string values
#othtypes[-4] = "int"

# FEC Record Number - unique row ID
othtypes[-1] = "int"
dateCols = ['transaction_date']
interCommitteeTransactionsDtype = zip(sqlFriendlyNames(interCommitteeTransactions.columns), othtypes)
ictdf = dd.read_csv(fullDownloadDir + "/interCommitteeTransactions*.csv*", sep="|", 
                    dtype=interCommitteeTransactionsDtype, parse_dates=dateCols,
                    error_bad_lines=False)


pas2types = ["str"]*len(committeeToCandidate.columns)
# Transaction Amount
pas2types[-8] = "float"

# File Number / Report ID - raw data contains string values
#pas2types[-4] = "int"

# FEC Record Number - unique row ID
pas2types[-1] = "int"
dateCols = ['transaction_date']
committeeToCandidateDtype = zip(sqlFriendlyNames(committeeToCandidate.columns), pas2types)
ctcdf = dd.read_csv(fullDownloadDir + "/committeeToCandidate*.csv*", sep="|", 
                    dtype=committeeToCandidateDtype, parse_dates=dateCols,
                    error_bad_lines=False)


indivtypes = ["str"]*len(individualContributions.columns)
# Transaction Amount
indivtypes[-7] = "float"

# File Number / Report ID - raw data contains string values
#indivtypes[-4] = "int"

# FEC Record Number - unique row ID - raw data contains string values
#indivtypes[-1] = "int"
dateCols = ['transaction_date']
individualContributionsDtype = zip(sqlFriendlyNames(individualContributions.columns), indivtypes)
indf = dd.read_csv(fullDownloadDir + "/individualContributions*.csv*", sep="|", 
                   dtype=individualContributionsDtype, parse_dates=dateCols,
                   error_bad_lines=False)


oppexptypes = ["str"]*len(operatingExpenditures.columns)
oppexptypes[2] = "int"
# Transaction Amount
oppexptypes[-13] = "float"

# FEC Record Number - unique row ID - raw data contains string values
# oppexptypes[-5] = "int"

# File Number / Report ID - raw data contains string values
#oppexptypes[-4] = "int"
dateCols = ['transaction_date']
operatingExpendituresDtype = zip(sqlFriendlyNames(operatingExpenditures.columns), oppexptypes)
oxdf = dd.read_csv(fullDownloadDir + "/operatingExpenditures*.csv*", sep="|", 
                   dtype=operatingExpendituresDtype, parse_dates=dateCols,
                   error_bad_lines=False, na_values=["O", "P", "G", "R" "C"])


In [64]:
indf.head(2)

,Filer Identification Number,Amendment Indicator,Report Type,Primary-General Indicator,Image Number,Transaction Type,Entity Type,Contributor/Lender/Transfer Name,City/Town,State,...,Employer,Occupation,Transaction Date(MMDDYYYY),Transaction Amount,Other Identification Number,Transaction ID,File Number / Report ID,Memo Code,Memo Text,FEC Record Number
0,C00078279,A,M11,P,80031492155,22Y,NaN,"MCKENNON, K R",MIDLAND,MI,...,NaN,NaN,10031979,400.0,NaN,NaN,NaN,NaN,CONTRIBUTION REF TO INDIVIDUAL,3062020110011466469
1,C00078279,A,M11,NaN,79031415137,15,NaN,"OREFFICE, P",MIDLAND,MI,...,DOW CHEMICAL CO,NaN,10261979,1500.0,NaN,NaN,NaN,NaN,NaN,3061920110000382948


In [ ]:
indf.groupby('Employer')['Transaction Amount'].sum().nlargest(25).compute()

## Complete data dictionary

##### Committee Master File
<table width="100%" border="0" cellpadding="3" cellspacing="2">
    <tbody><tr align="left" valign="middle" bgcolor="#004477">
      <td width="19%" height="25" nowrap=""><strong><font color="white">Column Name  </font></strong></td>
      <td width="26%" height="25"><font color="white"><span class="style3">Field Name </span></font></td>
      <td width="9%"><strong><font color="white">Position</font></strong></td>
      <td width="6%"><strong><font color="white">Null?</font></strong></td>
      <td width="11%" height="25"><font color="white"><span class="style3"> Data Type </span></font></td>
      <td width="29%" height="25"><font color="white"><span class="style3"> Description </span></font></td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CMTE_ID<br>      </td>
      <td align="center" valign="middle">Committee Identification</td>
      <td align="center" valign="middle">1</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">VARCHAR2(9)<br>      </td>
      <td> A 9-character alpha-numeric code assigned to a committee by the Federal Election Commission. The committee ID for a specific committee always remains the same. </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CMTE_NM<br></td>
      <td align="center" valign="middle">Committee Name</td>
      <td align="center" valign="middle">2</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(200)<br></td>
      <td>&nbsp; </td>
    </tr>
    
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">TRES_NM<br></td>
      <td align="center" valign="middle">Treasurer's Name<br></td>
      <td align="center" valign="middle">3</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(90)<br></td>
      <td> The officially registered treasurer for the committee. </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td height="47" align="center" valign="middle">CMTE_ST1<br></td>
      <td align="center" valign="middle">Street One</td>
      <td align="center" valign="middle">4</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(34)<br></td>
      <td>&nbsp;      </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CMTE_ST2<br></td>
      <td align="center" valign="middle">Street Two<br></td>
      <td align="center" valign="middle">5</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(34)<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CMTE_CITY<br></td>
      <td align="center" valign="middle">City or Town<br></td>
      <td align="center" valign="middle">6</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(30)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CMTE_ST<br></td>
      <td align="center" valign="middle">State<br></td>
      <td align="center" valign="middle">7</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(2)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CMTE_ZIP<br></td>
      <td align="center" valign="middle">Zip Code<br></td>
      <td align="center" valign="middle">8</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(9)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CMTE_DSGN<br></td>
      <td align="center" valign="middle">Committee Designation<br></td>
      <td align="center" valign="middle">9</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(1)<br></td>
      <td>A = Authorized by a candidate<br>
        B = Lobbyist/Registrant PAC<br>
        D = Leadership PAC<br>
        J = Joint fundraiser<br>
        P = Principal campaign committee of a candidate<br>
      U = Unauthorized</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CMTE_TP<br></td>
      <td align="center" valign="middle">Committee Type<br></td>
      <td align="center" valign="middle">10</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(1)<br></td>
      <td><a href="../metadata/CommitteeTypeCodes.shtml">List of committee type codes</a></td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle"> CMTE_PTY_AFFILIATION<br></td>
      <td align="center" valign="middle"> Committee Party<br></td>
      <td align="center" valign="middle">11</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle"> VARCHAR2(3)<br></td>
      <td><a href="../metadata/DataDictionaryPartyCodeDescriptions.shtml">List of party codes</a></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CMTE_FILING_FREQ<br></td>
      <td align="center" valign="middle">Filing Frequency<br></td>
      <td align="center" valign="middle">12</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(1)<br></td>
      <td>A = Administratively terminated<br>
        D = Debt<br>
        M = Monthly filer<br>
        Q = Quarterly filer<br>
        T = Terminated<br>
      W = Waived </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">ORG_TP<br></td>
      <td align="center" valign="middle">Interest Group Category<br></td>
      <td align="center" valign="middle">13</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(1)<br></td>
      <td>C = Corporation<br>
        L = Labor organization<br>
        M = Membership organization<br>
        T = Trade association<br>
        V = Cooperative<br>
      W = Corporation without capital stock </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CONNECTED_ORG_NM<br></td>
      <td align="center" valign="middle">Connected Organization's Name<br></td>
      <td align="center" valign="middle">14</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(200)<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CAND_ID<br></td>
      <td align="center" valign="middle">Candidate Identification<br></td>
      <td align="center" valign="middle">15</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(9)<br></td>
      <td>When a committee has a committee type designation of H, S, or P the identification number of the candidate will be entered in this field. </td>
    </tr>
  </tbody></table>
  
##### Candidate Master File
<table width="100%" border="0" cellpadding="3" cellspacing="2">
    <tbody><tr align="left" valign="middle" bgcolor="#004477">
      <td width="19%" height="25" nowrap=""><strong><font color="white">Column Name  </font></strong></td>
      <td width="26%" height="25"><font color="white"><span class="style3">Field Name </span></font></td>
      <td width="9%"><strong><font color="white">Position</font></strong></td>
      <td width="6%"><strong><font color="white">Null?</font></strong></td>
      <td width="11%" height="25"><font color="white"><span class="style3"> Data Type </span></font></td>
      <td width="29%" height="25"><font color="white"><span class="style3"> Description </span></font></td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CAND_ID<br>      </td>
      <td align="center" valign="middle">Candidate Identification<br></td>
      <td align="center" valign="middle">1</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">VARCHAR2(9)<br>      </td>
      <td> A 9-character alpha-numeric code assigned to a candidate by the Federal Election Commission. The candidate ID for a specific candidate remains the same across election cycles as long as the candidate is running for the same office.</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CAND_NAME<br></td>
      <td align="center" valign="middle">Candidate Name<br></td>
      <td align="center" valign="middle">2</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VVARCHAR2(200)<br></td>
      <td>&nbsp; </td>
    </tr>
    
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CAND_PTY_AFFILIATION<br></td>
      <td align="center" valign="middle">Party Affiliation<br></td>
      <td align="center" valign="middle">3</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(3)<br></td>
      <td> The political party affiliation reported by the candidate. </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td height="47" align="center" valign="middle">CAND_ELECTION_YR<br></td>
      <td align="center" valign="middle">Year of Election<br></td>
      <td align="center" valign="middle">4</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">Number(4)<br></td>
      <td>Candidate's election year from a Statement of Candidacy or state ballot list. </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CAND_OFFICE_ST<br></td>
      <td align="center" valign="middle">Candidate State<br></td>
      <td align="center" valign="middle">5</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(2)<br></td>
      <td><p style="margin-top: 0; margin-bottom: 0"> House = state of race </p>
        <p style="margin-top: 0; margin-bottom: 0;">President&nbsp; = US </p>
      <p style="margin-top: 0; margin-bottom: 0;">Senate = state of race </p></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CAND_OFFICE<br></td>
      <td align="center" valign="middle">Candidate Office<br></td>
      <td align="center" valign="middle">6</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(1)<br></td>
      <td><p style="margin-top: 0; margin-bottom: 0">H = House</p>
      <p style="margin-top: 0; margin-bottom: 0;">P = President</p>
      <p style="margin-top: 0; margin-bottom: 0;">S = Senate  </p></td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CAND_OFFICE_DISTRICT<br></td>
      <td align="center" valign="middle">Candidate District<br></td>
      <td align="center" valign="middle">7</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(2)<br></td>
      <td><p style="margin-top: 0; margin-bottom: 0">Congressional district number.</p>
      <p style="margin-top: 0; margin-bottom: 0;">Congressional At Large 00 </p>
      <p style="margin-top: 0; margin-bottom: 0;">Senate 00</p>
      <p style="margin-top: 0; margin-bottom: 0;">Presidential 00 </p></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CAND_ICI<br></td>
      <td align="center" valign="middle">Incumbent Challenger Status<br></td>
      <td align="center" valign="middle">8</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(1)<br></td>
      <td>C = Challenger<br>
        I = Incumbent<br>
      O = Open Seat is used to indicate an open seat.&nbsp; Open seats are defined as seats where the incumbent never sought re-election. </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CAND_STATUS<br></td>
      <td align="center" valign="middle">Candidate Status<br></td>
      <td align="center" valign="middle">9</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(1)<br></td>
      <td>C = Statutory candidate<br>
        F = Statutory candidate for future election<br>
        N = Not yet a statutory candidate<br>
      P = Statutory candidate in prior cycle</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CAND_PCC<br></td>
      <td align="center" valign="middle">Principal Campaign Committee<br></td>
      <td align="center" valign="middle">10</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(9)<br></td>
      <td> The ID assigned by the Federal Election Commission to the candidate's principal campaign committee for a given election cycle. </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle"> CAND_ST1<br></td>
      <td align="center" valign="middle"> Mailing Address - Street<br></td>
      <td align="center" valign="middle">11</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle"> VARCHAR2(34)<br></td>
      <td align="center"><div align="left">Mailing Address - Street </div></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CAND_ST2<br></td>
      <td align="center" valign="middle">Mailing Address - Street2 <br></td>
      <td align="center" valign="middle">12</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(34)<br></td>
      <td align="center"><div align="left">Mailing Address - Street2</div></td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td height="26" align="center" valign="middle">CAND_CITY<br></td>
      <td align="center" valign="middle">Mailing Address - City <br></td>
      <td align="center" valign="middle">13</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(30)<br></td>
      <td align="center"><div align="left">Mailing Address - City </div></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CAND_ST<br></td>
      <td align="center" valign="middle">Mailing Address - State <br></td>
      <td align="center" valign="middle">14</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(2)<br></td>
      <td align="center"><div align="left">Mailing Address - State </div></td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CAND_ZIP<br></td>
      <td align="center" valign="middle">Mailing Address - Zip Code <br></td>
      <td align="center" valign="middle">15</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2(9)<br></td>
      <td><div align="left">Mailing Address - Zip Code </div></td>
    </tr>
  </tbody></table>
  
##### Candidate to Committee Linkage File
<table width="100%" border="0" cellpadding="3" cellspacing="2">
    <tbody><tr align="left" valign="middle" bgcolor="#004477">
      <td width="19%" height="25" nowrap=""><strong><font color="white">Column Name  </font></strong></td>
      <td width="26%" height="25"><font color="white"><span class="style3">Field Name </span></font></td>
      <td width="6%"><strong><font color="white">Position</font></strong></td>
      <td width="5%"><strong><font color="white">Null?</font></strong></td>
      <td width="16%" height="25"><font color="white"><span class="style3"> Data Type </span></font></td>
      <td width="28%" height="25"><font color="white"><span class="style3"> Description </span></font></td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CAND_ID<br>      </td>
      <td align="center" valign="middle">Candidate Identification<br></td>
      <td align="center" valign="middle">1</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">VARCHAR2 (9)<br>      </td>
      <td> A 9-character alpha-numeric code assigned to a candidate by the Federal Election Commission. The candidate ID for a specific candidate remains the same across election cycles as long as the candidate is running for the same office.</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td height="31" align="center" valign="middle">CAND_ELECTION_YR<br></td>
      <td align="center" valign="middle">Candidate Election Year<br></td>
      <td align="center" valign="middle">2</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">NUMBER (4)<br></td>
      <td>Candidate's election year </td>
    </tr>
    
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">FEC_ELECTION_YR<br></td>
      <td align="center" valign="middle">FEC Election Year<br></td>
      <td align="center" valign="middle">3</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">NUMBER (4)<br></td>
      <td>Active 2-year period </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td height="47" align="center" valign="middle">CMTE_ID<br></td>
      <td align="center" valign="middle">Committee Identification<br></td>
      <td align="center" valign="middle">4</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (9)<br></td>
      <td>A 9-character alpha-numeric code assigned to a committee by the Federal Election Commission. The committee ID for a specific committee always remains the same. </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CMTE_TP<br></td>
      <td align="center" valign="middle">Committee Type<br></td>
      <td align="center" valign="middle">5</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (1)<br></td>
      <td><a href="http://www.fec.gov/finance/disclosure/metadata/CommitteeTypeCodes.shtml">List of committee type codes</a> </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CMTE_DSGN<br></td>
      <td align="center" valign="middle">Committee Designation<br></td>
      <td align="center" valign="middle">6</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (1)<br></td>
      <td>A = Authorized by a candidate<br>
B = Lobbyist/Registrant PAC<br>
D = Leadership PAC<br>
J = Joint fundraiser<br>
P = Principal campaign committee of a candidate<br>
U = Unauthorized</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">LINKAGE_ID<br></td>
      <td align="center" valign="middle">Linkage ID<br></td>
      <td align="center" valign="middle">7</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">NUMBER (12)<br></td>
      <td> Unique link ID </td>
    </tr>
  </tbody></table>
  
  
##### Contributions to Committees from Committees File
<table width="100%" border="0" cellpadding="3" cellspacing="2">
    <tbody><tr align="left" valign="middle" bgcolor="#004477">
      <td width="19%" height="25" nowrap=""><strong><font color="white">Column Name  </font></strong></td>
      <td width="26%" height="25"><font color="white"><span class="style3">Field Name </span></font></td>
      <td width="9%"><strong><font color="white">Position</font></strong></td>
      <td width="6%"><strong><font color="white">Null?</font></strong></td>
      <td width="11%" height="25"><font color="white"><span class="style3"> Data Type </span></font></td>
      <td width="29%" height="25"><font color="white"><span class="style3"> Description </span></font></td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CMTE_ID<br>        </td>
      <td align="center" valign="middle">Filer Identification Number<br></td>
      <td align="center" valign="middle">1</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">VARCHAR2 (9)<br>      </td>
      <td> A 9-character alpha-numeric code assigned to a committee by the Federal Election Commission </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">AMNDT_IND</td>
      <td align="center" valign="middle">Amendment Indicator<br></td>
      <td align="center" valign="middle">2</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (1)<br></td>
      <td> Indicates if the report being filed is new (N), an amendment (A) to a previous report, or a termination (T) report. </td>
    </tr>
    
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">RPT_TP</td>
      <td align="center" valign="middle">Report Type<br></td>
      <td align="center" valign="middle">3</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (3)<br></td>
      <td> Indicates the type of report filed. 


      <a href="http://www.fec.gov/finance/disclosure/metadata/ReportTypeCodes.shtml">List of report type codes</a> </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td height="47" align="center" valign="middle">TRANSACTION_PGI</td>
      <td align="center" valign="middle">Primary-General Indicator<br></td>
      <td align="center" valign="middle">4</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (5)<br></td>
      <td> <p>This code indicates the  election for which the contribution was made. EYYYY (election Primary, General, Other plus election year) </p>
      </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">IMAGE_NUM<br></td>
      <td align="center" valign="middle">Image Number<br></td>
      <td align="center" valign="middle">5</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (11)<br>
      or VARCHAR2(18)</td>
      <td>11-digit Image Number Format<br>
        YYOORRRFFFF<br>
        YY - scanning year <br>
        OO - office (01 - House, 02 - Senate, 03 - FEC Paper, 9x - FEC Electronic) <br>
        RRR - reel number <br>
        FFFF- frame number<br>
        <p>18-digit Image Number Format (June 29, 2015)<br>
          YYYYMMDDSSPPPPPPPP<br>
          YYYY - scanning year <br>
          MM - scanning month <br>
          DD - scanning day <br>
          SS - source (02 - Senate, 03 - FEC Paper, 9x - FEC Electronic) <br>
      PPPPPPPP - page (reset to zero every year on January 1) </p></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">TRANSACTION_TP<br></td>
      <td align="center" valign="middle">Transaction Type<br></td>
      <td align="center" valign="middle">6</td>
      <td align="center" valign="middle">Y</td>
<td align="center" valign="middle">VARCHAR2 (3)<br></td>
      <td><p style="margin-top: 0">Transaction types 10J, 11J, 13, 15J, 15Z, 16C, 16F, 16G, 16R, 17R, 17Z, 18G, 18J, 18K, 18U, 19J, 20, 20C, 20F, 20G, 20R, 22H, 22Z, 23Y, 24A, 24C, 24E, 24F, 24G, 24H, 24K, 24N, 24P, 24R, 24U, 24Z and 29 are included in the OTH file. Beginning with 2016 transaction types 30F, 30G, 30J, 30K, 31F, 31G, 31J, 31K, 32F, 32G, 32J, 32K, 40, 40Z, 41, 41Z, 42 and 42Z are also included in the OTH file.</p>
        <p style="margin-bottom: 0;">For more information about transaction type codes see this <a href="http://www.fec.gov/finance/disclosure/metadata/DataDictionaryTransactionTypeCodes.shtml">list of transaction type codes</a> </p>
      </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">ENTITY_TP<br></td>
      <td align="center" valign="middle">Entity Type<br></td>
      <td align="center" valign="middle">7</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (3)<br></td>
      <td>ONLY VALID FOR ELECTRONIC FILINGS received after April 2002.
        <p>CAN = Candidate<br>
          CCM = Candidate Committee<br>
          COM = Committee<br>
          IND = Individual (a person)<br>
          ORG = Organization (not a committee and not a person)<br>
          PAC = Political Action Committee<br>
      PTY = Party Organization</p></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">NAME<br></td>
      <td align="center" valign="middle">Contributor/Lender/Transfer Name<br></td>
      <td align="center" valign="middle">8</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (200)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CITY<br></td>
      <td align="center" valign="middle">City/Town<br></td>
      <td align="center" valign="middle">9</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (30)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">STATE<br></td>
      <td align="center" valign="middle">State<br></td>
      <td align="center" valign="middle">10</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (2)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle"> ZIP_CODE<br></td>
      <td align="center" valign="middle"> Zip Code<br></td>
      <td align="center" valign="middle">11</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle"> VARCHAR2 (9)<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">EMPLOYER<br></td>
      <td align="center" valign="middle">Employer<br></td>
      <td align="center" valign="middle">12</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (38)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">OCCUPATION<br></td>
      <td align="center" valign="middle">Occupation<br></td>
      <td align="center" valign="middle">13</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (38)<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">TRANSACTION_DT<br></td>
      <td align="center" valign="middle">Transaction Date(MMDDYYYY)<br></td>
      <td align="center" valign="middle">14</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">DATE<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">TRANSACTION_AMT<br></td>
      <td align="center" valign="middle">Transaction Amount<br></td>
      <td align="center" valign="middle">15</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">NUMBER (14,2)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">OTHER_ID<br></td>
      <td align="center" valign="middle">Other Identification Number<br></td>
      <td align="center" valign="middle">16</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (9)<br></td>
      <td>For contributions from individuals this column is null. For contributions from candidates or other committees this column will contain that contributor's FEC ID.</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">TRAN_ID<br></td>
      <td align="center" valign="middle">Transaction ID<br></td>
      <td align="center" valign="middle">17</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (32)<br></td>
      <td>ONLY VALID FOR ELECTRONIC FILINGS. A unique identifier associated with each itemization or transaction appearing in an FEC electronic file.  A transaction ID is unique for a specific committee for a specific report. In other words, if committee, C1, files a Q3 New with transaction SA123 and then files 3 amendments to the Q3 transaction SA123 will be identified by transaction ID SA123 in all 4 filings.</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">FILE_NUM<br></td>
      <td align="center" valign="middle">File Number / Report ID<br></td>
      <td align="center" valign="middle">18</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">NUMBER (22)<br></td>
      <td>Unique report id </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">MEMO_CD<br></td>
      <td align="center" valign="middle">Memo Code<br></td>
      <td align="center" valign="middle">19</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (1)<br></td>
      <td>'X' indicates that the amount is NOT to be included in the itemization total.  </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">MEMO_TEXT<br></td>
      <td align="center" valign="middle">Memo Text<br></td>
      <td align="center" valign="middle">20</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (100)<br></td>
      <td>A description of the activity. Memo Text is available on itemized amounts on Schedules A and B. These transactions are included in the itemization total.  </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">SUB_ID</td>
      <td align="center" valign="middle">FEC Record Number</td>
      <td align="center" valign="middle">21</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">NUMBER (19)</td>
      <td>Unique row ID </td>
    </tr>
  </tbody></table>
  
##### Contributions to Candidates from Committees File

<table width="100%" border="0" cellpadding="3" cellspacing="2">
    <tbody><tr align="left" valign="middle" bgcolor="#004477">
      <td width="19%" height="25" nowrap=""><strong><font color="white">Column Name  </font></strong></td>
      <td width="26%" height="25"><font color="white"><span class="style3">Field Name </span></font></td>
      <td width="9%"><strong><font color="white">Position</font></strong></td>
      <td width="6%"><strong><font color="white">Null?</font></strong></td>
      <td width="11%" height="25"><font color="white"><span class="style3"> Data Type </span></font></td>
      <td width="29%" height="25"><font color="white"><span class="style3"> Description </span></font></td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CMTE_ID<br>        </td>
      <td align="center" valign="middle">Filer Identification Number<br></td>
      <td align="center" valign="middle">1</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">VARCHAR2 (9)<br>      </td>
      <td> Identification number of committee filing report. A 9-character alpha-numeric code assigned to a committee by the Federal Election Commission </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">AMNDT_IND</td>
      <td align="center" valign="middle">Amendment Indicator<br></td>
      <td align="center" valign="middle">2</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (1)<br></td>
      <td> Indicates if the report being filed is new (N), an amendment (A) to a previous report, or a termination (T) report. </td>
    </tr>
    
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">RPT_TP</td>
      <td align="center" valign="middle">Report Type<br></td>
      <td align="center" valign="middle">3</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (3)<br></td>
      <td> Indicates the type of report filed. 


      <a href="http://www.fec.gov/finance/disclosure/metadata/ReportTypeCodes.shtml">List of report type codes</a></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td height="47" align="center" valign="middle">TRANSACTION_PGI</td>
      <td align="center" valign="middle">Primary-General Indicator<br></td>
      <td align="center" valign="middle">4</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (5)<br></td>
      <td> <p>This code indicates the  election for which the contribution was made. EYYYY (election Primary, General, Other plus election year) </p>
      </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">IMAGE_NUM<br></td>
      <td align="center" valign="middle">Image Number<br></td>
      <td align="center" valign="middle">5</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (11) or VARCHAR2(18)<br></td>
      <td>11-digit Image Number Format<br>
        YYOORRRFFFF<br>
        YY - scanning year <br>
        OO - office (01 - House, 02 - Senate, 03 - FEC Paper, 9x - FEC Electronic) <br>
        RRR - reel number <br>
        FFFF- frame number<br>
        <p>18-digit Image Number Format (June 29, 2015)<br>
          YYYYMMDDSSPPPPPPPP<br>
          YYYY - scanning year <br>
          MM - scanning month <br>
          DD - scanning day <br>
          SS - source (02 - Senate, 03 - FEC Paper, 9x - FEC Electronic) <br>
      PPPPPPPP - page (reset to zero every year on January 1) </p></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">TRANSACTION_TP<br></td>
      <td align="center" valign="middle">Transaction Type<br></td>
      <td align="center" valign="middle">6</td>
      <td align="center" valign="middle">Y</td>
<td align="center" valign="middle">VARCHAR2 (3)<br></td>
      <td><p style="margin-top: 0">Transaction types 24A, 24C, 24E, 24F, 24H, 24K, 24N, 24P, 24R and 24Z are included in the PAS2 file.</p>
        <p style="margin-bottom: 0;">For more information about transaction type codes see this <a href="http://www.fec.gov/finance/disclosure/metadata/DataDictionaryTransactionTypeCodes.shtml">list of transaction type codes</a> </p>
      </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">ENTITY_TP<br></td>
      <td align="center" valign="middle">Entity Type<br></td>
      <td align="center" valign="middle">7</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (3)<br></td>
      <td>ONLY VALID FOR ELECTRONIC FILINGS received after April 2002.
        <p>CAN = Candidate<br>
          CCM = Candidate Committee<br>
          COM = Committee<br>
          IND = Individual (a person)<br>
          ORG = Organization (not a committee and not a person)<br>
          PAC = Political Action Committee<br>
      PTY = Party Organization</p></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">NAME<br></td>
      <td align="center" valign="middle">Recipient/Payee<br></td>
      <td align="center" valign="middle">8</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (200)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CITY<br></td>
      <td align="center" valign="middle">City/Town<br></td>
      <td align="center" valign="middle">9</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (30)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">STATE<br></td>
      <td align="center" valign="middle">State<br></td>
      <td align="center" valign="middle">10</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (2)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle"> ZIP_CODE<br></td>
      <td align="center" valign="middle"> Zip Code<br></td>
      <td align="center" valign="middle">11</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle"> VARCHAR2 (9)<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">EMPLOYER<br></td>
      <td align="center" valign="middle">Employer<br></td>
      <td align="center" valign="middle">12</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (38)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">OCCUPATION<br></td>
      <td align="center" valign="middle">Occupation<br></td>
      <td align="center" valign="middle">13</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (38)<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">TRANSACTION_DT<br></td>
      <td align="center" valign="middle">Transaction Date(MMDDYYYY)<br></td>
      <td align="center" valign="middle">14</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">DATE<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">TRANSACTION_AMT<br></td>
      <td align="center" valign="middle">Transaction Amount<br></td>
      <td align="center" valign="middle">15</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">NUMBER (14,2)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">OTHER_ID<br></td>
      <td align="center" valign="middle">Other Identification Number<br></td>
      <td align="center" valign="middle">16</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (9)<br></td>
      <td>FEC ID of the recipient committee or the supported or opposed candidate ID.</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CAND_ID</td>
      <td align="center" valign="middle">Candidate Identification Number</td>
      <td align="center" valign="middle">17</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (9)</td>
      <td> Recipient Candidate ID. Candidate receiving money from the filing committee </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">TRAN_ID<br></td>
      <td align="center" valign="middle">Transaction ID<br></td>
      <td align="center" valign="middle">18</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (32)<br></td>
      <td>ONLY VALID FOR ELECTRONIC FILINGS. A unique identifier associated with each itemization or transaction appearing in an FEC electronic file.  A transaction ID is unique for a specific committee for a specific report. In other words, if committee, C1, files a Q3 New with transaction SA123 and then files 3 amendments to the Q3 transaction SA123 will be identified by transaction ID SA123 in all 4 filings.</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">FILE_NUM<br></td>
      <td align="center" valign="middle">File Number / Report ID<br></td>
      <td align="center" valign="middle">19</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">NUMBER (22)<br></td>
      <td>Unique report id </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">MEMO_CD<br></td>
      <td align="center" valign="middle">Memo Code<br></td>
      <td align="center" valign="middle">20</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (1)<br></td>
      <td>'X' indicates that the amount is NOT to be included in the itemization total.  </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">MEMO_TEXT<br></td>
      <td align="center" valign="middle">Memo Text<br></td>
      <td align="center" valign="middle">21</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (100)<br></td>
      <td>A description of the activity. Memo Text is available on itemized amounts on Schedules A and B. These transactions are included in the itemization total.  </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">SUB_ID</td>
      <td align="center" valign="middle">FEC Record Number</td>
      <td align="center" valign="middle">22</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">NUMBER (19)</td>
      <td>Unique row ID </td>
    </tr>
  </tbody></table>
  
  
##### Contributions by Individuals File

<table width="100%" border="0" cellpadding="3" cellspacing="2">
    <tbody><tr align="left" valign="middle" bgcolor="#004477">
      <td width="19%" height="25" nowrap=""><strong><font color="white">Column Name  </font></strong></td>
      <td width="26%" height="25"><font color="white"><span class="style3">Field Name </span></font></td>
      <td width="9%"><strong><font color="white">Position</font></strong></td>
      <td width="6%"><strong><font color="white">Null?</font></strong></td>
      <td width="11%" height="25"><font color="white"><span class="style3"> Data Type </span></font></td>
      <td width="29%" height="25"><font color="white"><span class="style3"> Description </span></font></td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CMTE_ID<br>        </td>
      <td align="center" valign="middle">Filer Identification Number<br></td>
      <td align="center" valign="middle">1</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">VARCHAR2 (9)<br>      </td>
      <td> A 9-character alpha-numeric code assigned to a committee by the Federal Election Commission </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">AMNDT_IND</td>
      <td align="center" valign="middle">Amendment Indicator<br></td>
      <td align="center" valign="middle">2</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (1)<br></td>
      <td> Indicates if the report being filed is new (N), an amendment (A) to a previous report, or a termination (T) report. </td>
    </tr>
    
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">RPT_TP</td>
      <td align="center" valign="middle">Report Type<br></td>
      <td align="center" valign="middle">3</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (3)<br></td>
      <td>Indicates the type of report filed. <a href="http://www.fec.gov/finance/disclosure/metadata/ReportTypeCodes.shtml">List of report type codes</a> </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td height="47" align="center" valign="middle">TRANSACTION_PGI</td>
      <td align="center" valign="middle">Primary-General Indicator<br></td>
      <td align="center" valign="middle">4</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (5)<br></td>
      <td> This code indicates the election for which the contribution was made. EYYYY (election plus election year)<br>
	P = Primary<br>
	G = General<br>
	O = Other<br>
	C = Convention<br>
	R = Runoff<br>
	S = Special<br>
	E = Recount<br>
      </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">IMAGE_NUM<br></td>
      <td align="center" valign="middle">Image Number<br></td>
      <td align="center" valign="middle">5</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (11) or VARCHAR2(18)<br></td>
      <td>11-digit Image Number Format<br>
        YYOORRRFFFF<br>
        YY - scanning year <br>
        OO - office (01 - House, 02 - Senate, 03 - FEC Paper, 9x - FEC Electronic) <br>
        RRR - reel number <br>
        FFFF- frame number<br>
        <p>18-digit Image Number Format (June 29, 2015)<br>
          YYYYMMDDSSPPPPPPPP<br>
          YYYY - scanning year <br>
          MM - scanning month <br>
          DD - scanning day <br>
          SS - source (02 - Senate, 03 - FEC Paper, 9x - FEC Electronic) <br>
      PPPPPPPP - page (reset to zero every year on January 1) </p></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">TRANSACTION_TP<br></td>
      <td align="center" valign="middle">Transaction Type<br></td>
      <td align="center" valign="middle">6</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (3)<br></td>
      <td><p style="margin-top: 0;">Transaction types 10, 11, 15, 15C, 15E, 15I, 15T, 19, 22Y, 24I, 24T, 20Y and 21Y are included in the INDIV file. Beginning with 2016 transaction types 30, 30T, 31, 31T, 32, 32T, 40T, 40Y, 41T, 41Y, 42T and 42Y are also included in the INDIV file. </p>
        <p style="margin-bottom: 0;">For more information about transaction type codes see this <a href="http://www.fec.gov/finance/disclosure/metadata/DataDictionaryTransactionTypeCodes.shtml">list of transaction type codes</a> </p>
      </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">ENTITY_TP<br></td>
      <td align="center" valign="middle">Entity Type<br></td>
      <td align="center" valign="middle">7</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (3)<br></td>
      <td>ONLY VALID FOR ELECTRONIC FILINGS received after April 2002.
        <p>CAN = Candidate<br>
          CCM = Candidate Committee<br>
          COM = Committee<br>
          IND = Individual (a person)<br>
          ORG = Organization (not a committee and not a person)<br>
          PAC = Political Action Committee<br>
      PTY = Party Organization</p></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">NAME<br></td>
      <td align="center" valign="middle">Contributor/Lender/Transfer Name<br></td>
      <td align="center" valign="middle">8</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (200)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CITY<br></td>
      <td align="center" valign="middle">City/Town<br></td>
      <td align="center" valign="middle">9</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (30)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">STATE<br></td>
      <td align="center" valign="middle">State<br></td>
      <td align="center" valign="middle">10</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (2)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle"> ZIP_CODE<br></td>
      <td align="center" valign="middle"> Zip Code<br></td>
      <td align="center" valign="middle">11</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle"> VARCHAR2 (9)<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">EMPLOYER<br></td>
      <td align="center" valign="middle">Employer<br></td>
      <td align="center" valign="middle">12</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (38)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">OCCUPATION<br></td>
      <td align="center" valign="middle">Occupation<br></td>
      <td align="center" valign="middle">13</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (38)<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">TRANSACTION_DT<br></td>
      <td align="center" valign="middle">Transaction Date(MMDDYYYY)<br></td>
      <td align="center" valign="middle">14</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">DATE<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">TRANSACTION_AMT<br></td>
      <td align="center" valign="middle">Transaction Amount<br></td>
      <td align="center" valign="middle">15</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">NUMBER (14,2)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">OTHER_ID<br></td>
      <td align="center" valign="middle">Other Identification Number<br></td>
      <td align="center" valign="middle">16</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (9)<br></td>
      <td>For contributions from individuals this column is null. For contributions from candidates or other committees this column will contain that contributor's FEC ID.</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">TRAN_ID<br></td>
      <td align="center" valign="middle">Transaction ID<br></td>
      <td align="center" valign="middle">17</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (32)<br></td>
      <td>ONLY VALID FOR ELECTRONIC FILINGS. A unique identifier associated with each itemization or transaction appearing in an FEC electronic file.  A transaction ID is unique for a specific committee for a specific report. In other words, if committee, C1, files a Q3 New with transaction SA123 and then files 3 amendments to the Q3 transaction SA123 will be identified by transaction ID SA123 in all 4 filings.</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">FILE_NUM<br></td>
      <td align="center" valign="middle">File Number / Report ID<br></td>
      <td align="center" valign="middle">18</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">NUMBER (22)<br></td>
      <td>Unique report id </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">MEMO_CD<br></td>
      <td align="center" valign="middle">Memo Code<br></td>
      <td align="center" valign="middle">19</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (1)<br></td>
      <td>'X' indicates that the amount is NOT to be included in the itemization total.  </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">MEMO_TEXT<br></td>
      <td align="center" valign="middle">Memo Text<br></td>
      <td align="center" valign="middle">20</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (100)<br></td>
      <td>A description of the activity. Memo Text is available on itemized amounts on Schedules A and B. These transactions are included in the itemization total.  </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">SUB_ID</td>
      <td align="center" valign="middle">FEC Record Number</td>
      <td align="center" valign="middle">21</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">NUMBER (19)</td>
      <td>Unique row ID </td>
    </tr>
  </tbody></table>
  
  
##### Operating Expenditures File

<table width="100%" border="0" cellpadding="3" cellspacing="2">
    <tbody><tr align="left" valign="middle" bgcolor="#004477">
      <td width="19%" height="25" nowrap=""><strong><font color="white">Column Name  </font></strong></td>
      <td width="26%" height="25"><font color="white"><span class="style3">Field Name </span></font></td>
      <td width="9%"><strong><font color="white">Position</font></strong></td>
      <td width="6%"><strong><font color="white">Null?</font></strong></td>
      <td width="11%" height="25"><font color="white"><span class="style3"> Data Type </span></font></td>
      <td width="29%" height="25"><font color="white"><span class="style3"> Description </span></font></td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CMTE_ID<br>        </td>
      <td align="center" valign="middle">Filer Identification Number<br></td>
      <td align="center" valign="middle">1</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">VARCHAR2 (9)<br>      </td>
      <td>Identification number of committee filing report. A 9-character alpha-numeric code assigned to a committee by the Federal Election Commission </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">AMNDT_IND</td>
      <td align="center" valign="middle">Amendment Indicator<br></td>
      <td align="center" valign="middle">2</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (1)<br></td>
      <td> Indicates if the report being filed is new (N), an amendment (A) to a previous report, or a termination (T) report. </td>
    </tr>
    
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">RPT_YR</td>
      <td align="center" valign="middle">Report Year <br></td>
      <td align="center" valign="middle">3</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">Number(4)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td height="47" align="center" valign="middle">RPT_TP</td>
      <td align="center" valign="middle">Report Type<br></td>
      <td align="center" valign="middle">4</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (3)</td>
      <td> <br>
Indicates the type of report filed. <a href="http://www.fec.gov/finance/disclosure/metadata/ReportTypeCodes.shtml">List of report type codes</a>      </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">IMAGE_NUM<br></td>
      <td align="center" valign="middle">Image Number<br></td>
      <td align="center" valign="middle">5</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (11) or VARCHAR2(18)<br></td>
      <td>11-digit Image Number Format<br>
        YYOORRRFFFF<br>
        YY - scanning year <br>
        OO - office (01 - House, 02 - Senate, 03 - FEC Paper, 9x - FEC Electronic) <br>
        RRR - reel number <br>
        FFFF- frame number<br>
        <p>18-digit Image Number Format (June 29, 2015)<br>
          YYYYMMDDSSPPPPPPPP<br>
          YYYY - scanning year <br>
          MM - scanning month <br>
          DD - scanning day <br>
          SS - source (02 - Senate, 03 - FEC Paper, 9x - FEC Electronic) <br>
      PPPPPPPP - page (reset to zero every year on January 1) </p></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">LINE_NUM<br></td>
      <td align="center" valign="middle">Line Number <br></td>
      <td align="center" valign="middle">6</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle"><br></td>
      <td><p style="margin-top: 0;">Indicates FEC Form line number </p>
      </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">FORM_TP_CD<br></td>
      <td align="center" valign="middle">Form Type <br></td>
      <td align="center" valign="middle">7</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (8)<br></td>
      <td>Indicates FEC Form</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">SCHED_TP_CD<br></td>
      <td align="center" valign="middle">Schedule Type<br></td>
      <td align="center" valign="middle">8</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (8)<br></td>
      <td> Schedule B - Itemized Disbursements </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">NAME<br></td>
      <td align="center" valign="middle">Contributor/Lender/Transfer Name<br></td>
      <td align="center" valign="middle">9</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (200)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CITY<br></td>
      <td align="center" valign="middle">City/Town<br></td>
      <td align="center" valign="middle">10</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (30)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle"> STATE<br></td>
      <td align="center" valign="middle"> State<br></td>
      <td align="center" valign="middle">11</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle"> VARCHAR2 (2)<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">ZIP_CODE<br></td>
      <td align="center" valign="middle">Zip Code<br></td>
      <td align="center" valign="middle">12</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (9)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">TRANSACTION_DT<br></td>
      <td align="center" valign="middle">Transaction Date(MMDDYYYY)<br></td>
      <td align="center" valign="middle">13</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">DATE<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">TRANSACTION_AMT<br></td>
      <td align="center" valign="middle">Transaction Amount<br></td>
      <td align="center" valign="middle">14</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">NUMBER (14,2)<br></td>
      <td>&nbsp; </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">TRANSACTION_PGI<br></td>
      <td align="center" valign="middle">Primary General Indicator</td>
      <td align="center" valign="middle">15</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (5)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">PURPOSE<br></td>
      <td align="center" valign="middle">Purpose<br></td>
      <td align="center" valign="middle">16</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (100)<br></td>
      <td>&nbsp;</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">CATEGORY<br></td>
      <td align="center" valign="middle">Disbursement Category Code <br></td>
      <td align="center" valign="middle">17</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (3)<br></td>
      <td>001-012 and 101-107 </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">CATEGORY_DESC<br></td>
      <td align="center" valign="middle">Disbursement Category Code Description <br></td>
      <td align="center" valign="middle">18</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (40)<br></td>
      <td>List of <a href="CategoryCodes.shtml">Disbursement Category Codes</a> and their meaning</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">MEMO_CD<br></td>
      <td align="center" valign="middle">Memo Code<br></td>
      <td align="center" valign="middle">19</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (1)<br></td>
      <td>'X' indicates that the amount is NOT to be included in the itemization total. </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">MEMO_TEXT<br></td>
      <td align="center" valign="middle">Memo Text<br></td>
      <td align="center" valign="middle">20</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (100)<br></td>
      <td>A description of the activity. Memo Text is available on itemized amounts on Schedule B. These transactions are included in the itemization total. </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">ENTITY_TP</td>
      <td align="center" valign="middle">Entity Type </td>
      <td align="center" valign="middle">21</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (3)</td>
      <td>ONLY VALID FOR ELECTRONIC FILINGS received after April 2002.
        <p>CAN = Candidate<br>
          CCM = Candidate Committee<br>
          COM = Committee<br>
          IND = Individual (a person)<br>
          ORG = Organization (not a committee and not a person)<br>
          PAC = Political Action Committee<br>
      PTY = Party Organization </p></td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">SUB_ID<br></td>
      <td align="center" valign="middle"> FEC Record Number<br></td>
      <td align="center" valign="middle">22</td>
      <td align="center" valign="middle">N</td>
      <td align="center" valign="middle">NUMBER (19)<br></td>
      <td>Unique row ID </td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">FILE_NUM </td>
      <td align="center" valign="middle">File Number / Report ID  </td>
      <td align="center" valign="middle">23</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">NUMBER (7) </td>
      <td>Unique report id </td>
    </tr>
    <tr valign="middle" bgcolor="#FFFFFF">
      <td align="center" valign="middle">TRAN_ID<br></td>
      <td align="center" valign="middle"> Transaction ID<br></td>
      <td align="center" valign="middle">24</td>
      <td align="center" valign="middle">&nbsp;</td>
      <td align="center" valign="middle">VARCHAR2 (32) <br></td>
      <td>ONLY VALID FOR ELECTRONIC FILINGS. A unique identifier associated with each itemization or transaction appearing in an FEC electronic file.  A transaction ID is unique for a specific committee for a specific report. In other words, if committee, C1, files a Q3 New with transaction SA123 and then files 3 amendments to the Q3 transaction SA123 will be identified by transaction ID SA123 in all 4 filings.</td>
    </tr>
    <tr valign="middle" bgcolor="#F5F0FF">
      <td align="center" valign="middle">BACK_REF_TRAN_ID </td>
      <td align="center" valign="middle">Back Reference Transaction ID  </td>
      <td align="center" valign="middle">25</td>
      <td align="center" valign="middle">Y</td>
      <td align="center" valign="middle">VARCHAR2 (32)  </td>
      <td>ONLY VALID FOR ELECTRONIC FILINGS. Used to associate one transaction with another transaction in the same report (using file number, transaction ID and back reference transaction ID). For example, a credit card payment and the subitemization of specific purchases. The back reference transaction ID of the specific purchases will equal the transaction ID of the payment to the credit card company. </td>
    </tr>
  </tbody></table>
  
##### Party Codes

<table width="100%" border="0" cellpadding="3" cellspacing="2">
    <tbody>
    <tr bgcolor="#336699">
      <th width="16%" height="30" scope="col"><font color="#FFFFFF">Party Code  </font></th>
      <th width="40%" scope="col"><font color="#FFFFFF">Party Code  Description</font></th>
      <th width="44%" scope="col"><font color="#FFFFFF">Notes</font></th>
    </tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	ACE	</div></td><td scope="row">	Ace Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	AKI	</div></td><td nowrap="" scope="row">	Alaskan Independence Party	</td>
<td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	AIC	</div></td><td scope="row">	American Independent Conservative	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	AIP	</div></td><td scope="row">	American Independent Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	AMP	</div></td><td scope="row">	American Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	APF	</div></td><td scope="row">	American People's Freedom Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	AE	</div></td><td scope="row">	Americans Elect	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	CIT	</div></td><td scope="row">	Citizens' Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	CMD	</div></td><td scope="row">	Commandments Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	CMP	</div></td><td scope="row">	Commonwealth Party of the U.S.	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	COM	</div></td><td scope="row">	Communist Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	CNC	</div></td><td scope="row">	Concerned Citizens Party Of Connecticut	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	CRV	</div></td><td scope="row">	Conservative Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	CON	</div></td><td scope="row">	Constitution Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	CST	</div></td><td scope="row">	Constitutional	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	COU	</div></td><td scope="row">	Country	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	DCG	</div></td><td scope="row">	D.C. Statehood Green Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	DNL	</div></td><td scope="row">	Democratic -Nonpartisan League	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	DEM	</div></td><td scope="row">	Democratic Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	D/C	</div></td><td scope="row">	Democratic/Conservative	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	DFL	</div></td><td scope="row">	Democratic-Farmer-Labor	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	DGR	</div></td><td scope="row">	Desert Green Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	FED	</div></td><td scope="row">	Federalist	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	FLP	</div></td><td scope="row">	Freedom Labor Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	FRE	</div></td><td scope="row">	Freedom Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	GWP	</div></td><td scope="row">	George Wallace Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	GRT	</div></td><td scope="row">	Grassroots	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	GRE	</div></td><td scope="row">	Green Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	GR	</div></td><td scope="row">	Green-Rainbow	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	HRP	</div></td><td scope="row">	Human Rights Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	IDP	</div></td><td scope="row">	Independence Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	IND	</div></td><td scope="row">	Independent	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	IAP	</div></td><td scope="row">	Independent American Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	ICD	</div></td><td scope="row">	Independent Conservative Democratic	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	IGR	</div></td><td scope="row">	Independent Green	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	IP	</div></td><td scope="row">	Independent Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	IDE	</div></td><td scope="row">	Independent Party of Delaware	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	IGD	</div></td><td scope="row">	Industrial Government Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	JCN	</div></td><td scope="row">	Jewish/Christian National	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	JUS	</div></td><td scope="row">	Justice Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	LRU	</div></td><td scope="row">	La Raza Unida 	</td><td scope="row">	Also see RUP	</td></tr>
<tr><td scope="row"><div align="left">	LBR	</div></td><td scope="row">	Labor Party	</td><td scope="row">	Also see LAB	</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	LFT	</div></td><td scope="row">	Less Federal Taxes	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	LBL	</div></td><td scope="row">	Liberal Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	LIB	</div></td><td scope="row">	Libertarian Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	LBU	</div></td><td scope="row">	Liberty Union Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	MTP	</div></td><td scope="row">	Mountain Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	NDP	</div></td><td scope="row">	National Democratic Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	NLP	</div></td><td scope="row">	Natural Law Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	NA	</div></td><td scope="row">	New Alliance	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	NJC	</div></td><td scope="row">	New Jersey Conservative Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	NPP	</div></td><td scope="row">	New Progressive Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	NPA	</div></td><td scope="row">	No Party Affiliation	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	NOP	</div></td><td scope="row">	No Party Preference	</td><td scope="row">	Commonly used in CA &amp; WA	</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	NNE	</div></td><td scope="row">	None	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	N	</div></td><td scope="row">	Nonpartisan	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	NON	</div></td><td scope="row">	Non-Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	OE	</div></td><td scope="row">	One Earth Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	OTH	</div></td><td scope="row">	Other	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	PG	</div></td><td scope="row">	Pacific Green	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	PSL	</div></td><td scope="row">	Party for Socialism and Liberation	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	PAF	</div></td><td scope="row">	Peace And Freedom	</td><td scope="row">	Also see PFP	</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	PFP	</div></td><td scope="row">	Peace And Freedom Party	</td><td scope="row">	Also see PAF	</td></tr>
<tr><td scope="row"><div align="left">	PFD	</div></td><td scope="row">	Peace Freedom Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	POP	</div></td><td scope="row">	People Over Politics	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	PPY	</div></td><td scope="row">	People's Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	PCH	</div></td><td scope="row">	Personal Choice Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	PPD	</div></td><td scope="row">	Popular Democratic Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	PRO	</div></td><td scope="row">	Progressive Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	NAP	</div></td><td scope="row">	Prohibition Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	PRI	</div></td><td scope="row">	Puerto Rican Independence Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	RUP	</div></td><td scope="row">	Raza Unida Party	</td><td scope="row">	Also see LRU	</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	REF	</div></td><td scope="row">	Reform Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	REP	</div></td><td scope="row">	Republican Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	RES	</div></td><td scope="row">	Resource Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	RTL	</div></td><td scope="row">	Right To Life	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	SEP	</div></td><td scope="row">	Socialist Equality Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	SLP	</div></td><td scope="row">	Socialist Labor Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	SUS	</div></td><td scope="row">	Socialist Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	SOC	</div></td><td scope="row">	Socialist Party U.S.A.	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	SWP	</div></td><td scope="row">	Socialist Workers Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	TX	</div></td><td scope="row">	Taxpayers	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	TWR	</div></td><td scope="row">	Taxpayers Without Representation 	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	TEA	</div></td><td scope="row">	Tea Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	THD	</div></td><td scope="row">	Theo-Democratic	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	LAB	</div></td><td scope="row">	U.S. Labor Party	</td><td scope="row">	Also see LBR	</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	USP	</div></td><td scope="row">	U.S. People's Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	UST	</div></td><td scope="row">	U.S. Taxpayers Party	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	UN	</div></td><td scope="row">	Unaffiliated	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	UC	</div></td><td scope="row">	United Citizen	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	UNI	</div></td><td scope="row">	United Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	UNK	</div></td><td scope="row">	Unknown	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	VET	</div></td><td scope="row">	Veterans Party	</td><td scope="row">		</td></tr>
<tr><td scope="row"><div align="left">	WTP	</div></td><td scope="row">	We the People	</td><td scope="row">		</td></tr>
<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">	W	</div></td><td scope="row">	Write-In	</td><td scope="row">		</td></tr>    

    </tbody><tbody>
  </tbody></table>

##### FEC Committee Type Codes

<table width="100%" border="0" cellpadding="2" cellspacing="2">
    <tbody><tr align="left" bgcolor="#004477">
      <td height="35"><b><font color="white"><strong>Committee</strong><span class="style3"> Type Code </span></font></b></td>
      <td height="35"><b><font color="white"><strong>Committee</strong><span class="style3"> Type </span></font></b></td>
      <td height="35"><b><font color="white"><span class="style3">Explanation</span></font></b></td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td width="13%" height="35" align="left">C</td>
      <td width="18%" height="35">Communication Cost </td>
      <td width="69%" height="35">Organizations like corporations or unions may prepare communications for their employees or members that advocate the election of specific candidates and they must disclose them under certain circumstances. These are usually paid with direct corporate or union funds rather than from PACs.</td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">D</td>
      <td height="35">Delegate Committee </td>
      <td height="35"> Delegate  committees are organized for the purpose of influencing the selection of delegates to Presidential nominating conventions. The term includes a group of delegates, a group of individuals seeking to become delegates, and a group of individuals supporting delegates. </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">E</td>
      <td height="35">Electioneering Communication </td>
      <td height="35">Groups (other than PACs) making <a href="http://www.fec.gov/finance/disclosure/electioneering.shtml">Electioneering Communications </a></td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">H</td>
      <td height="35">House</td>
      <td height="35">Campaign committees for candidates for the House of Representatives </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">I</td>
      <td height="35">Independent Expenditor (Person or Group)</td>
      <td height="35">Individuals or groups (other than PACs) making <a href="http://www.fec.gov/pages/brochures/indexp.shtml#IE">independent expenditures</a> over $250 in a year must disclose those expenditures </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">N</td>
      <td height="35">PAC - Nonqualified </td>
      <td height="35">PACs that have not yet been in existence for six months and received contributions from 50 people and made contributions to five federal candidates. These committees have lower limits for their contributions to candidates. </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">O</td>
      <td height="35">Independent Expenditure-Only (Super PACs)</td>
      <td height="35">Political Committee that has filed a statement consistent with <a href="http://saos.fec.gov/saos/searchao?AONUMBER=2010-09">AO 2010-09</a>
	or <a href="http://saos.fec.gov/saos/searchao?AONUMBER=2010-11">AO 2010-11.</a>  For more information about <a href="http://www.fec.gov/pages/brochures/indexp.shtml#IE">independent expenditures</a></td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">P</td>
      <td height="35">Presidential</td>
      <td height="35">Campaign committee for candidate for President </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">Q</td>
      <td height="35">PAC - Qualified </td>
      <td height="35">PACs that have  been in existence for six months and received contributions from 50 people and made contributions to five federal candidates</td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">S</td>
      <td height="35">Senate</td>
      <td height="35">Campaign committee for candidate for Senate </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">U</td>
      <td height="35">Single Candidate Independent Expenditure</td>
      <td height="35">Political Committee For more information about <a href="http://www.fec.gov/pages/brochures/indexp.shtml#IE">independent expenditures</a></td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">V</td>
      <td height="35">PAC with Non-Contribution Account - Nonqualified</td>
      <td height="35"><a href="http://www.fec.gov/press/press2011/20111006postcarey.shtml">Political committees with non-contribution accounts</a></td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">W</td>
      <td height="35">PAC with Non-Contribution Account - Qualified</td>
      <td height="35"><a href="http://www.fec.gov/press/press2011/20111006postcarey.shtml">Political committees with non-contribution accounts</a></td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">X</td>
      <td height="35">Party - Nonqualified </td>
      <td height="35">Party committees that have not yet been in existence for six months and received contributions from 50 people, unless they are affiliated with another party committee that has met these requirements. </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">Y</td>
      <td height="35">Party - Qualified </td>
      <td height="35">Party committees that have existed for at least six months and received contributions from 50 people or are affiliated with another party committee that meets these requirements. </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">Z</td>
      <td height="35">National Party Nonfederal Account </td>
      <td height="35">National party nonfederal accounts. Not permitted after enactment of Bipartisan Campaign Reform Act of 2002. </td>
    </tr>

  </tbody></table>
  
##### Disbursement Category Codes

<table width="100%" border="0" cellpadding="2" cellspacing="2">
    <tbody><tr align="left" bgcolor="#004477">
      <td width="13%" height="35"><b><font color="white"><strong>Disbursment Category </strong><span class="style3">Code </span></font></b></td>
      <td width="69%" height="35"><b><font color="white"><strong>Disbursment Category </strong><span class="style3">Cod</span></font></b><font color="white"><strong>e Description</strong></font></td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" colspan="2" align="left"><div align="center">Disbursement Category Codes for non-presidential filers </div></td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">001</td>
      <td height="35">Administrative/Salary/Overhead Expenses (e.g., rent, staff salaries, postage, office supplies,<br>
      equipment, furniture, ballot access fees, petition drives, party fees and legal and accounting expenses)</td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">002</td>
      <td height="35">Travel Expenses - including travel reimbursement expenses<br>
      (e.g., costs of commercial carrier tickets; reimbursements for use of private vehicles, advance payments for use of corporate aircraft; lodging and meal expenses incurred during travel)</td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">003</td>
      <td height="35">Solicitation and Fundraising Expenses (e.g., costs for direct mail solicitations and fundraising events including printing, mailing lists, consultant fees, call lists, invitations, catering costs and room rental)</td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">004</td>
      <td height="35">Advertising Expenses -including general public political advertising (e.g., purchases of radio/television broadcast/cable time, print advertisements and related production costs)</td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">005</td>
      <td height="35">Polling Expenses</td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">006</td>
      <td height="35">Campaign Materials (e.g., buttons, bumper stickers, brochures, mass mailings, pens, posters, balloons)</td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">007</td>
      <td height="35">Campaign Event Expenses (e.g., costs associated with candidate appearances, campaign rallies, town meetings, phone banks, including catering costs, door to door get-out-the-vote efforts and driving voters to the polls)</td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">008</td>
      <td height="35">Transfers (e.g., to other authorized committees of the same candidate)</td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">009</td>
      <td height="35">Loan Repayments (e.g., repayments of loans made/guaranteed by the candidate or other person)</td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">010</td>
      <td height="35">Refunds of Contributions (e.g., contribution refunds to individuals/ persons, political party committees or other political committees)</td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">011</td>
      <td height="35">Political Contributions (e.g., contributions to other federal candidates and committees, donations to nonfederal candidates and committees)</td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">012</td>
      <td height="35">Donations (e.g., donations to charitable or civic organizations</td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" colspan="2" align="left"><div align="center">Disbursement Category Codes for presidential filers</div></td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">101</td>
      <td height="35">Expenses that are not allocable</td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">102</td>
      <td height="35">Media Expenditures</td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">103</td>
      <td height="35">Expenditures for Mass Mailings and other Campaign Materials (e.g., buttons, bumper stickers, brochures, mass mailings, pens, posters, balloons)</td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">104</td>
      <td height="35">Overhead Expenditures of State Offices and their Facilities (e.g., rent, staff salaries, postage, office supplies, equipment, furniture, ballot access fees, petition drives, party fees and legal and accounting expenses)</td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">105</td>
      <td height="35">Expenditures for Special Telephone Programs</td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">106</td>
      <td height="35">Public Opinion Poll Expenditures</td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">107</td>
      <td height="35">Fundraising Expenditures (e.g., costs for direct mail solicitations &amp; fundraising events including printing, mailing lists, consultant fees, call lists, invitations, catering costs and room rental)</td>
    </tr>
  </tbody></table>
  
  
##### Report Type Codes and their Meaning

<table width="100%" border="0" cellpadding="2" cellspacing="2">
    <tbody><tr align="left" bgcolor="#004477">
      <td height="35"><b><font color="white"><span class="style3">Report Type Code </span></font></b></td>
      <td height="35"><b><font color="white"><span class="style3">Report Type </span></font></b></td>
      <td height="35"><b><font color="white"><span class="style3">Explanation</span></font></b></td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td width="13%" height="35" align="left">12C</td>
      <td width="18%" height="35">PRE-CONVENTION</td>
      <td width="69%" height="35">For states using conventions to select candidates. Report covers through 20 days before the convention </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">12G</td>
      <td height="35">PRE-GENERAL</td>
      <td height="35">Report covers through 20 days before the general election - due 12 days before the election </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">12P</td>
      <td height="35">PRE-PRIMARY</td>
      <td height="35">Report covers through 20 days before the primary- due 12 days before the election </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">12R</td>
      <td height="35">PRE-RUN-OFF</td>
      <td height="35">Report covers through 20 days before the run-off- due 12 days before the election </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">12S</td>
      <td height="35">PRE-SPECIAL</td>
      <td height="35">Report covers through 20 days before the special election - due 12 days before the election </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">30D</td>
      <td height="35">POST-ELECTION</td>
      <td height="35">Report covers from 19 days before the election through 20 days after - due 30 days after the election </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">30G</td>
      <td height="35">POST-GENERAL</td>
      <td height="35">Report covers from 19 days before the election through 20 days after. - due 30 days after the election </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">30P</td>
      <td height="35">POST-PRIMARY</td>
      <td height="35">Report covers from 19 days before the election through 20 days after. - due 30 days after the election </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">30R</td>
      <td height="35">POST-RUN-OFF</td>
      <td height="35">Report covers from 19 days before the election through 20 days after. - due 30 days after the election </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">30S</td>
      <td height="35">POST-SPECIAL</td>
      <td height="35">Report covers from 19 days before the election through 20 days after. - due 30 days after the election </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">60D</td>
      <td height="35">POST-CONVENTION</td>
      <td height="35">Report filed by national party convention and host committees disclosing their convention expenses, due 60 days after the convention </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">ADJ</td>
      <td height="35">COMP ADJUST AMEND</td>
      <td height="35">Adjustment of a comprehensive amendment - coverage is variable </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">CA</td>
      <td height="35">COMPREHENSIVE AMEND</td>
      <td height="35">Amendment modifying information from two or more original reports - coverage is variable </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">M10</td>
      <td height="35">OCTOBER MONTHLY</td>
      <td height="35">Covers September - due October 20 </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">M11</td>
      <td height="35">NOVEMBER MONTHLY</td>
      <td height="35">Covers October - due November 20 </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">M12</td>
      <td height="35">DECEMBER MONTHLY</td>
      <td height="35">Covers November - due December 20 </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">M2</td>
      <td height="35">FEBRUARY MONTHLY</td>
      <td height="35">Covers January - due February 20 </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">M3</td>
      <td height="35">MARCH MONTHLY</td>
      <td height="35">Covers February - due March 20 </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">M4</td>
      <td height="35">APRIL MONTHLY</td>
      <td height="35">Covers March - due April 20 </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">M5</td>
      <td height="35">MAY MONTHLY</td>
      <td height="35">Covers April - due May 20 </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">M6</td>
      <td height="35">JUNE MONTHLY</td>
      <td height="35">Covers May - due June 20 </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">M7</td>
      <td height="35">JULY MONTHLY</td>
      <td height="35">Covers June - due July 20 </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">M8</td>
      <td height="35">AUGUST MONTHLY</td>
      <td height="35">Covers July - due August 20 </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">M9</td>
      <td height="35">SEPTEMBER MONTHLY</td>
      <td height="35">Covers August - due September 20 </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">MY</td>
      <td height="35">MID-YEAR REPORT</td>
      <td height="35">Covers January 1 through June 30 - due July 31 Permissible in non-election years for PACs and party committees normally filing Quarterly reports. (Note that since 2003 campaign committees must file quarterly in all years.) </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">Q1</td>
      <td height="35">APRIL QUARTERLY</td>
      <td height="35">Covers January 1 through March 31 - due April 15 </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">Q2</td>
      <td height="35">JULY QUARTERLY</td>
      <td height="35">Covers April 1 through June 30 - due July 15 </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">Q3</td>
      <td height="35">OCTOBER QUARTERLY</td>
      <td height="35">Covers July 1 through September 30 - due October 15 </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">TER</td>
      <td height="35">TERMINATION REPORT</td>
      <td height="35">Final report submitted by a committee - coverage is variable </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">YE</td>
      <td height="35">YEAR-END</td>
      <td height="35">Covers from the end of the last quarterly or mid-year report through December 31 - due January 31 </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">90S</td>
      <td height="35">POST INAUGURAL SUPPLEMENT</td>
      <td height="35">&nbsp;</td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">90D</td>
      <td height="35">POST INAUGURAL</td>
      <td height="35">Filing of Presidential inaugural committee - due 90 days after the Inauguration </td>
    </tr>
    <tr bgcolor="#F5F0FF">
      <td height="35" align="left">48H</td>
      <td height="35">48 HOUR NOTIFICATION</td>
      <td height="35">Report of specific contribution of $1,000 or more made to a campaign within 20 days of an election. Alternatively, once a PAC or party or other person has made independent expenditures exceeding $10,000 in a race these and future independent expenditures must be reported. Due within 48 hours of receiving the contribution or public distribution of the independent expenditure. 48 hour timing for independent expenditures applies prior to 20 days before the election. </td>
    </tr>
    <tr bgcolor="#FFFFFF">
      <td height="35" align="left">24H</td>
      <td height="35">24 HOUR NOTIFICATION</td>
      <td height="35">Within 20 days of an election once a PAC or party or other person has made independent expenditures exceeding $1,000 in a race these and future independent expenditures must be reported. Due within 24 hours of the public distribution of the independent expenditure. </td>
    </tr>
  </tbody></table>
  
  
##### Transaction Type Codes and their meaning

<table width="100%" border="0" cellpadding="3" cellspacing="2">
    <tbody>
    <tr bgcolor="#336699">
      <th width="16%" height="30" scope="col"><font color="#FFFFFF"> Transaction Type</font></th>
      <th width="84%" scope="col"><font color="#FFFFFF">Transaction Type Description</font></th>
    </tr>
    
<tr><td scope="row"><div align="left">10</div></td>
<td scope="row">Contribution to Independent Expenditure-Only Committees (Super PACs), Political Committees with non-contribution accounts (Hybrid PACs) and nonfederal party "soft money" accounts (1991-2002) from a person (individual, partnership, limited liability company, corporation, labor organization, or any other organization or group of persons)</td></tr>

<tr bgcolor="#F5F0FF">
  <td>10J</td>
  <td scope="row">Memo - Recipient committee's percentage of nonfederal receipt  from a person (individual, partnership, limited liability company, corporation, labor organization, or any other organization or group of persons)</td>
</tr>
<tr><td scope="row"><div align="left">11</div></td>
<td scope="row">Native American Tribe contribution</td></tr>
<tr bgcolor="#F5F0FF">
  <td>11J</td>
  <td scope="row">Memo - Recipient committee's percentage of contribution from Native American Tribe given to joint fundraising committee</td>
</tr>

<tr><td scope="row"><div align="left">12</div></td>
<td scope="row">Nonfederal other receipt - Levin Account (Line 2) </td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">13</div></td>
<td scope="row">Inaugural donation accepted</td></tr>

<tr><td scope="row"><div align="left">15</div></td>
<td scope="row">Contribution to political committees (other than Super PACs and Hybrid PACs) from an individual, partnership or limited liability company</td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">15C</div></td>
<td scope="row">Contribution from candidate</td></tr>

<tr><td scope="row"><div align="left">15E</div></td>
<td scope="row">Earmarked contributions to political committees (other than Super PACs and Hybrid PACs) from an individual, partnership or limited liability company</td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">15F</div></td>
<td scope="row">Loans forgiven by candidate</td></tr>

<tr><td scope="row"><div align="left">15I</div></td>
<td scope="row">Earmarked contribution from an individual, partnership or limited liability company received by intermediary committee and passed on in the form of contributor's check (intermediary in)</td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">15J</div></td>
<td scope="row">Memo - Recipient committee's percentage of contribution from an individual, partnership or limited liability company given to joint fundraising committee</td></tr>

<tr><td scope="row"><div align="left">15T</div></td>
<td scope="row">Earmarked contribution from an individual, partnership or limited liability company received by intermediary committee and entered into intermediary's treasury (intermediary treasury in)</td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">15Z</div></td>
<td scope="row">In-kind contribution received from registered filer</td></tr>

<tr><td scope="row"><div align="left">16C</div></td>
<td scope="row">Loan received from the candidate</td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">16F</div></td>
<td scope="row">Loan received from bank</td></tr>

<tr><td scope="row"><div align="left">16G</div></td>
<td scope="row">Loan from individual</td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">16H</div></td>
<td scope="row">Loan from registered filers</td></tr>

<tr><td scope="row"><div align="left">16J</div></td>
<td scope="row">Loan repayment from individual</td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">16K</div></td>
<td scope="row">Loan repayment from from registered filer</td></tr>

<tr><td scope="row"><div align="left">16L</div></td>
<td scope="row">Loan repayment received from unregistered entity</td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">16R</div></td>
<td scope="row">Loan received from registered filers</td></tr>

<tr><td scope="row"><div align="left">16U</div></td>
<td scope="row">Loan received from unregistered entity</td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">17R</div></td>
<td scope="row">Contribution refund received from registered entity</td></tr>

<tr><td scope="row"><div align="left">17U</div></td>
<td scope="row">Refund/Rebate/Return received from unregistered entity</td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">17Y</div></td>
<td scope="row">Refund/Rebate/Return from individual or corporation</td></tr>

<tr><td scope="row"><div align="left">17Z</div></td>
<td scope="row">Refund/Rebate/Return from candidate or committee</td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">18G</div></td>
<td scope="row">Transfer in from affiliated committee</td></tr>

<tr><td scope="row"><div align="left">18H</div></td>
<td scope="row">Honorarium received</td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">18J</div></td>
<td scope="row">Memo - Recipient committee's percentage of contribution from a registered committee given to joint fundraising committee</td></tr>

<tr><td scope="row"><div align="left">18K</div></td>
<td scope="row">Contribution received from registered filer</td></tr>

<tr bgcolor="#F5F0FF"><td><div align="left">18L</div></td>
<td scope="row">Bundled contribution</td></tr>

<tr><td><div align="left">18U</div></td>
<td scope="row">Contribution received from unregistered committee</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">19</div></td>
<td scope="row">Electioneering communication donation received</td></tr>

<tr><td><div align="left">19J</div></td>
<td scope="row">Memo - Recipient committee's percentage of Electioneering Communication donation given to joint fundraising committee</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">20</div></td>
<td scope="row">Nonfederal disbursement -  nonfederal
 party "soft money" accounts (1991-2002)</td></tr>

<tr><td><div align="left">20A</div></td>
<td scope="row">Nonfederal disbursement - Levin Account (Line 4A) Voter Registration</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">20B</div></td>
<td scope="row">Nonfederal Disbursement - Levin Account (Line 4B) Voter Identification</td></tr>

<tr><td><div align="left">20C</div></td>
<td scope="row">Loan repayment made to candidate</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">20D</div></td>
<td scope="row">Nonfederal disbursement - Levin Account (Line 4D) Generic Campaign</td></tr>

<tr><td><div align="left">20F</div></td>
<td scope="row">Loan repayment made to banks</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">20G</div></td>
<td scope="row">Loan repayment made to individual</td></tr>

<tr><td><div align="left">20R</div></td>
<td scope="row">Loan repayment made to registered filer</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">20V</div></td>
<td scope="row">Nonfederal disbursement - Levin Account (Line 4C) Get Out The Vote</td></tr>

<tr>
  <td>20Y</td>
  <td scope="row">Nonfederal refund</td>
</tr>
<tr>
</tr><tr bgcolor="#F5F0FF"><td scope="row"><div align="left">21Y</div></td>
  <td scope="row">Native American Tribe refund</td>
</tr>
<tr><td><div align="left">22G</div></td>
<td scope="row">Loan to individual</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">22H</div></td>
<td scope="row">Loan to candidate or committee</td></tr>

<tr><td><div align="left">22J</div></td>
<td scope="row">Loan repayment to individual</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">22K</div></td>
<td scope="row">Loan repayment to candidate or committee</td></tr>

<tr><td><div align="left">22L</div></td>
<td scope="row">Loan repayment to bank</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">22R</div></td>
<td scope="row">Contribution refund to unregistered entity</td></tr>

<tr><td><div align="left">22U</div></td>
<td scope="row">Loan repaid to unregistered entity</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">22X</div></td>
<td scope="row">Loan made to unregistered entity</td></tr>

<tr><td><div align="left">22Y</div></td>
<td scope="row">Contribution refund to an individual, partnership or limited liability company</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">22Z</div></td>
<td scope="row">Contribution refund to candidate or committee</td></tr>

<tr><td><div align="left">23Y</div></td>
<td scope="row">Inaugural donation refund</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">24A</div></td>
<td scope="row">Independent expenditure opposing election of candidate</td></tr>

<tr><td><div align="left">24C</div></td>
<td scope="row">Coordinated party expenditure</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">24E</div></td>
<td scope="row">Independent expenditure advocating election of candidate</td></tr>

<tr><td><div align="left">24F</div></td>
<td scope="row">Communication cost for candidate (only for Form 7 filer)</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">24G</div></td>
<td scope="row">Transfer out to affiliated committee</td></tr>

<tr><td><div align="left">24H</div></td>
<td scope="row">Honorarium to candidate</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">24I</div></td>
<td scope="row">Earmarked contributor's check passed on by intermediary committee to intended recipient (intermediary out)</td></tr>

<tr><td><div align="left">24K</div></td>
<td scope="row">Contribution made to nonaffiliated committee</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">24N</div></td>
<td scope="row">Communication cost against candidate (only for Form 7 filer)</td></tr>

<tr><td><div align="left">24P</div></td>
<td scope="row">Contribution made to possible federal candidate including in-kind contributions</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">24R</div></td>
<td scope="row">Election recount disbursement</td></tr>

<tr><td><div align="left">24T</div></td>
<td scope="row">Earmarked contribution passed to intended recipient from intermediary's treasury (treasury out)</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">24U</div></td>
<td scope="row">Contribution made to unregistered entity</td></tr>

<tr><td><div align="left">24Z</div></td>
<td scope="row">In-kind contribution made to registered filer</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">28L</div></td>
<td scope="row">Refund of bundled contribution</td></tr>

<tr><td><div align="left">29</div></td>
<td scope="row">Electioneering Communication disbursement or obligation</td></tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">30</div></td>
  <td scope="row">Convention Account receipt from an individual, partnership or limited liability company</td>
</tr>

<tr>
  <td>30T</td>
  <td scope="row">Convention Account receipt from Native American Tribe</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">30K</div></td>
  <td scope="row">Convention Account receipt from registered filer</td>
</tr>

<tr>
  <td>30G</td>
  <td scope="row">Convention Account - transfer in from affiliated committee</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">30J</div></td>
  <td scope="row">Convention Account - Memo - Recipient committee's percentage of contributions from an individual, partnership or limited liability company given to joint fundraising committee</td>
</tr>

<tr>
  <td>30F</td>
  <td scope="row">Convention Account - Memo - Recipient committee's percentage of contributions from a registered committee given to joint fundraising committee</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">31</div></td>
  <td scope="row">Headquarters Account receipt from an individual, partnership or limited liability company</td>
</tr>

<tr>
  <td>31T</td>
  <td scope="row">Headquarters Account receipt from Native American Tribe</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">31K</div></td>
  <td scope="row">Headquarters Account receipt from registered filer</td>
</tr>

<tr>
  <td>31G</td>
  <td scope="row">Headquarters Account - transfer in from affiliated committee</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">31J</div></td>
  <td scope="row">Headquarters Account - Memo - Recipient committee's percentage of contributions from an individual, partnership or limited liability company given to joint fundraising committee</td>
</tr>

<tr>
  <td>31F</td>
  <td scope="row">Headquarters Account - Memo - Recipient committee's percentage of contributions from a registered committee given to joint fundraising committee</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">32</div></td>
  <td scope="row">Recount Account receipt from an individual, partnership or limited liability company</td>
</tr>

<tr>
  <td>32T</td>
  <td scope="row">Recount Account receipt from Native American Tribe</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">32K</div></td>
  <td scope="row">Recount Account receipt from registered filer</td>
</tr>

<tr>
  <td>32G</td>
  <td scope="row">Recount Account - transfer in from affiliated committee</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">32J</div></td>
  <td scope="row">Recount Account - Memo - Recipient committee's percentage of contributions from an individual, partnership or limited liability company given to joint fundraising committee</td>
</tr>

<tr>
  <td>32F</td>
  <td scope="row">Recount Account - Memo - Recipient committee's percentage of contributions from a registered committee given to joint fundraising committee</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">40</div></td>
  <td scope="row">Convention Account disbursement</td>
</tr>

<tr>
  <td>40Y</td>
  <td scope="row">Convention Account refund to  an individual, partnership or limited liability company</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">40T</div></td>
  <td scope="row">Convention Account refund to Native American Tribe</td>
</tr>

<tr>
  <td>40Z</td>
  <td scope="row">Convention Account refund to registered filer</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">41</div></td>
  <td scope="row">Headquarters Account disbursement</td>
</tr>

<tr>
  <td>41Y</td>
  <td scope="row">Headquarters Account refund to  an individual, partnership or limited liability company</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">41T</div></td>
  <td scope="row">Headquarters Account refund to Native American Tribe</td>
</tr>

<tr>
  <td>41Z</td>
  <td scope="row">Headquarters Account refund to registered filer</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">42</div></td>
  <td scope="row">Recount Account disbursement</td>
</tr>

<tr>
  <td>42Y</td>
  <td scope="row">Recount Account refund to  an individual, partnership or limited liability company</td>
</tr>

<tr bgcolor="#F5F0FF"><td scope="row"><div align="left">42T</div></td>
  <td scope="row">Recount Account refund to Native American Tribe</td>
</tr>

<tr>
  <td>42Z</td>
  <td scope="row">Recount Account refund to registered filer</td>
</tr>

    </tbody><tbody>
  </tbody></table>